In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Activation, Flatten, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
import re

# Step 1: Preprocess the Text
document = """The speed of transmission is an important point of difference between the two viruses. 
Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. 
The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. 

Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. 
In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. 

The reproductive number – the number of secondary infections generated from one infected individual – is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza. However, estimates for both COVID-19 and influenza viruses are very context and time-specific, making direct comparisons more difficult."""

# Clean and tokenize the text
document = document.lower()
document = re.sub(r'[^a-z\s]', '', document)  # Remove non-alphabetic characters

tokenizer = Tokenizer()
tokenizer.fit_on_texts([document])
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
index_word = tokenizer.index_word
sequences = tokenizer.texts_to_sequences([document])[0]

print(f"Vocabulary Size: {vocab_size}")
print(f"Word Index: {word_index}")
print(f"Tokenized Sequences: {sequences}")

# Step 2: Generate Context-Target Pairs for CBOW
window_size = 2  # Use context of 2 words before and after the target word
context = []
target = []



for i in range(window_size, len(sequences) - window_size):
    context.append([sequences[i - window_size], sequences[i + window_size]])
    target.append(sequences[i])

# Convert to numpy arrays
context = np.array(context)
target = np.array(target)

embedding_dim = 50  # Size of the word embedding
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=2))
model.add(SpatialDropout1D(0.2))
model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Step 4: Train the CBOW Model
context = np.array([context[:, 0], context[:, 1]]).T  # Reshape for context words
model.fit(context, target, epochs=100, verbose=1)

# Step 5: Predicting the Target Word from Context
def predict_word(context_words):
    # Convert context words to their corresponding indices
    context_indices = [word_index[word] for word in context_words]
    
    # Reshape context for the model (it expects a 2D array)
    context_input = np.array(context_indices).reshape(1, -1)
    
    # Predict the target word index
    predicted = model.predict(context_input)
    
    # Convert the predicted index to the corresponding word
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    predicted_word = index_word[predicted_word_index]
    
    return predicted_word

# Example Prediction - Predict target word for context words ["speed", "of"]
context_test = ["influenza", "viruses"]
predicted_word = predict_word(context_test)

print(f"Context: {context_test}")
print(f"Predicted Target Word: {predicted_word}")


Vocabulary Size: 93
Word Index: {'the': 1, 'of': 2, 'transmission': 3, 'influenza': 4, 'covid': 5, 'virus': 6, 'for': 7, 'is': 8, 'to': 9, 'a': 10, 'and': 11, 'between': 12, 'serial': 13, 'interval': 14, 'than': 15, 'be': 16, 'days': 17, 'are': 18, 'viruses': 19, 'shorter': 20, 'time': 21, 'from': 22, 'appearance': 23, 'symptoms': 24, 'while': 25, 'this': 26, 'that': 27, 'can': 28, 'in': 29, 'major': 30, 'driver': 31, 'number': 32, 'speed': 33, 'an': 34, 'important': 35, 'point': 36, 'difference': 37, 'two': 38, 'has': 39, 'median': 40, 'incubation': 41, 'period': 42, 'infection': 43, 'successive': 44, 'cases': 45, 'estimated': 46, 'means': 47, 'spread': 48, 'faster': 49, 'further': 50, 'first': 51, 'illness': 52, 'or': 53, 'potentially': 54, 'presymptomatic': 55, 'before': 56, 'contrast': 57, 'we': 58, 'learning': 59, 'there': 60, 'people': 61, 'who': 62, 'shed': 63, 'hours': 64, 'prior': 65, 'symptom': 66, 'onset': 67, 'at': 68, 'present': 69, 'does': 70, 'not': 71, 'appear': 72, 're

In [5]:
context_test = ["covid","influenza"]
predicted_word = predict_word(context_test)

print(f"Context: {context_test}")
print(f"Predicted Target Word: {predicted_word}")

1/1 [==============================] - 0s 20ms/step
Context: ['covid', 'influenza']
Predicted Target Word: transmission


In [6]:
embeddings = model.layers[0].get_weights()[0]
distance_matrix = np.sqrt(((embeddings[:, np.newaxis] - embeddings[np.newaxis, :]) ** 2).sum(axis=2))

# Step 5: Define function to find similar words using distance matrix
def find_similar_words(search_term, top_n=6):
    search_term_idx = word_index.get(search_term)
    
    if not search_term_idx:
        return f"'{search_term}' not found in the vocabulary."
    
    similar_indices = distance_matrix[search_term_idx - 1].argsort()[1:top_n + 1]
    similar_words = {search_term: [index_word[idx + 1] for idx in similar_indices]}
    
    return similar_words

# Example: Find similar words to 'influenza'
similar_words = find_similar_words('viruses')
print(f"Similar words to 'influenza': {similar_words}")

Similar words to 'influenza': {'viruses': ['faster', 'the', 'we', 'individual', 'one', 'both']}
